<a href="https://colab.research.google.com/github/GeorgeMarica/XLNET-text-classification/blob/main/XLNETcorrrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SentencePiece
!pip install transformers==4.10.0


     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import math
import numpy as np
from sklearn.metrics import classification_report
import torch.nn.functional as F
import torch
import os
import sentencepiece
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)

In [ ]:
df_data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/XLNET/data/train.csv", sep=",", encoding="utf-8", usecols=['user_suggestion', 'user_review'])
tokenizer = XLNetTokenizer(vocab_file='/content/gdrive/MyDrive/Colab Notebooks/XLNET/models/xlnet-base-cased/xlnet-base-cased-spiece.model', do_lower_case=False)
model_file_address = '/content/gdrive/MyDrive/Colab Notebooks/XLNET/models/xlnet-base-cased'
xlnet_out_address = '/content/gdrive/MyDrive/Colab Notebooks/XLNET/models/xlnet_out_model'

In [ ]:
tag2idx = {'0': 0, '1': 1}
max_len = 256
batch_num = 16
epochs = 4

In [ ]:
sentences = df_data.user_review.to_list()
labels = df_data.user_suggestion.to_list()
tag2name = {tag2idx[key]: key for key in tag2idx.keys()}
device = torch.device("cuda")

full_input_ids = []
full_input_masks = []
full_segment_ids = []

SEG_ID_A = 0
SEG_ID_B = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

UNK_ID = tokenizer.encode("<unk>")[0]
CLS_ID = tokenizer.encode("<cls>")[0]
SEP_ID = tokenizer.encode("<sep>")[0]
MASK_ID = tokenizer.encode("<mask>")[0]
EOD_ID = tokenizer.encode("<eod>")[0]

for i, sentence in enumerate(sentences):
    tokens_a = tokenizer.encode(sentence)   # Tokenize sentence to token id list
    if len(tokens_a) > max_len - 2:       # Trim the len of text
        tokens_a = tokens_a[:max_len - 2]
    tokens = []
    segment_ids = []
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(SEG_ID_A)
    tokens.append(SEP_ID)                   # Add <sep> token
    segment_ids.append(SEG_ID_A)
    tokens.append(CLS_ID)                   # Add <cls> token
    segment_ids.append(SEG_ID_CLS)
    input_ids = tokens
    input_mask = [0] * len(input_ids)       # The mask has 0 for real tokens and 1 for padding tokens. Only real tokens are attended to.
    if len(input_ids) < max_len:            # Zero-pad up to the sequence length at fornt
        delta_len = max_len - len(input_ids)
        input_ids = [0] * delta_len + input_ids
        input_mask = [1] * delta_len + input_mask
        segment_ids = [SEG_ID_PAD] * delta_len + segment_ids
    assert len(input_ids) == max_len
    assert len(input_mask) == max_len
    assert len(segment_ids) == max_len
    full_input_ids.append(input_ids)
    full_input_masks.append(input_mask)
    full_segment_ids.append(segment_ids)


tags = [tag2idx[str(lab)] for lab in labels]
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(full_input_ids, tags, full_input_masks,full_segment_ids,
                                                                                                  random_state=4, test_size=0.3)
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)


# Set token embedding, attention embedding, segment embedding
train_data = TensorDataset(tr_inputs, tr_masks,tr_segs, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks,val_segs, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [ ]:
model = XLNetForSequenceClassification.from_pretrained(model_file_address,num_labels=len(tag2idx))
model.to(device)
max_grad_norm = 1.0
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs
# Fine tune model all layer parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

model.train()
print("***** Running training *****")
print("  Num examples = %d" % (len(tr_inputs)))
print("  Batch size = %d" % (batch_num))
print("  Num steps = %d" % (num_train_optimization_steps))
for _ in trange(epochs, desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        print(step)
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segs, b_labels = batch
        # forward pass
        outputs = model(input_ids=b_input_ids, token_type_ids=b_segs, input_mask=b_input_mask, labels=b_labels)
        loss, logits = outputs[:2]
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss / nb_tr_steps))


model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join(xlnet_out_address, "pytorch_model.bin")
output_config_file = os.path.join(xlnet_out_address, "config.json")
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(xlnet_out_address)


Some weights of the model checkpoint at /content/gdrive/MyDrive/Colab Notebooks/XLNET/models/xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/Colab Notebooks/XLNET/models/xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.

***** Running training *****
  Num examples = 12245
  Batch size = 32
  Num steps = 1532


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Epoch:  25%|██▌       | 1/4 [11:40<35:00, 700.20s/it]

...

Epoch:  75%|███████▌  | 3/4 [35:00<11:40, 700.03s/it]

Train loss: 0.13373874326928942
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
26

Epoch: 100%|██████████| 4/4 [46:39<00:00, 699.81s/it]

Train loss: 0.09823485159014074


('/content/gdrive/MyDrive/Colab Notebooks/XLNET/models/xlnet_out_model/spiece.model',)

In [ ]:
model = XLNetForSequenceClassification.from_pretrained(xlnet_out_address,num_labels=len(tag2idx))
model.to(device)
model.eval()
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_predict = []
print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segs, b_labels = batch
    with torch.no_grad():
        outputs = model(input_ids=b_input_ids, token_type_ids=b_segs, input_mask=b_input_mask, labels=b_labels)
        tmp_eval_loss, logits = outputs[:2]
    # Get textclassification predict result
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    # Save predict and real label reuslt for analyze
    for predict in np.argmax(logits, axis=1):
        y_predict.append(predict)
    for real_result in label_ids.tolist():
        y_true.append(real_result)

report = classification_report(y_pred=np.array(y_predict), y_true=np.array(y_true))
print(report)

***** Running evaluation *****
  Num examples =5249
  Batch size = 32
              precision    recall  f1-score   support

           0       0.85      0.91      0.88      2212
           1       0.93      0.88      0.90      3037

    accuracy                           0.89      5249
   macro avg       0.89      0.90      0.89      5249
weighted avg       0.90      0.89      0.89      5249

